In [28]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler
from utils import neg_sample

import json
import math
import os
import random
from scipy.sparse import csr_matrix
from torch.utils.data import Dataset

from datasets import PretrainDataset
from models import S3RecModel
from trainers import PretrainTrainer
from utils import (
    EarlyStopping,
    check_path,
    get_item2attribute_json,
    get_user_seqs_long,
    set_seed,
)

In [43]:
# args.data_file = args.data_dir + args.data_name + '.txt'
data_file = "./data/train/train_ratings.csv"

# concat all user_seq get a long sequence, from which sample neg segment for SP
user_seq, max_item, long_sequence = get_user_seqs_long(data_file)
#item2attribute, attribute_size = get_item2attribute_json(item2attribute_file)

In [ ]:
item2attribute_file = "./data/train/genre_writer_item2attributes.json"

In [55]:
item2attribute = json.loads(open(item2attribute_file).readline())

attribute_set = set()
attribute_set2 = set()
for item, attributes in item2attribute.items():
    attribute_set = attribute_set | set(attributes[0])
    attribute_set2 = attribute_set2 | set(attributes[1])
attribute_size = max(attribute_set)
attribute_size2 = max(attribute_set2)

In [56]:
item2attribute

{'1': [[8, 12, 13, 5, 9], [296, 320, 748, 891, 2090, 2351, 2623]],
 '2': [[8, 13, 9], [1304, 2445, 2521]],
 '3': [[5, 6], [1435]],
 '4': [[5, 1, 6], [466]],
 '5': [[5], [1165, 1226, 1812, 2313]],
 '6': [[2, 0, 4], [79]],
 '7': [[5, 6], [104, 498, 1604, 2102, 2448]],
 '8': [[8, 13], [1663, 2353, 2512]],
 '9': [[2], [2078]],
 '10': [[2, 8, 4], [160, 659, 989, 1044]],
 '11': [[5, 1, 6], [2354]],
 '12': [[5, 10], [47, 849, 2397]],
 '13': [[8, 12, 13], [2256]],
 '14': [[1], [38, 2149, 2645]],
 '15': [[2, 8, 6], [479, 968, 1119, 1507, 1929]],
 '16': [[0, 1], [35, 2028]],
 '17': [[1, 6], [97, 115]],
 '18': [[5], [39, 209]],
 '19': [[5], [520, 1950]],
 '20': [[2, 5, 0, 1, 4], [1663, 2135]],
 '21': [[5, 0, 4], [191, 1050]],
 '22': [[0, 1, 10, 11, 4], [528]],
 '23': [[2, 0, 4], [173, 2549, 2550]],
 '24': [[1, 3], [2223]],
 '26': [[1], [96, 1983]],
 '27': [[13, 1], [1505]],
 '28': [[1, 6], [115]],
 '29': [[8, 1, 9, 11, 3], [70, 248, 371, 1583]],
 '31': [[1], [466]],
 '32': [[11, 3, 4], [287, 2006

In [54]:
len(attribute_set2)

2989

In [ ]:
item2attribute, attribute_size = get_item2attribute_json(item2attribute_file)

In [26]:
item_size = max_item + 2
mask_id = max_item + 1
attribute_size = attribute_size + 1

In [33]:
len(user_seq)

31360

In [34]:
31360*50

1568000

In [29]:
part_sequence = []
for seq in user_seq:
    input_ids = seq[-(50 + 2) : -2]  # keeping same as train set
    for i in range(len(input_ids)):
        part_sequence.append(input_ids[: i + 1])

In [35]:
part_sequence

[[5225],
 [5225, 1046],
 [5225, 1046, 64034],
 [5225, 1046, 64034, 53996],
 [5225, 1046, 64034, 53996, 43932],
 [5225, 1046, 64034, 53996, 43932, 26887],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319, 51698],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319, 51698, 5444],
 [5225, 1046, 64034, 53996, 43932, 26887, 66171, 68319, 51698, 5444, 6731],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455,
  56145],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455,
  56145,
  968],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  68319,
  51698,
  5444,
  6731,
  2455,
  56145,
  968,
  54995],
 [5225,
  1046,
  64034,
  53996,
  43932,
  26887,
  66171,
  

In [37]:
writers_df = pd.read_csv("./data/train/writers.tsv", sep="\t")
array, index = pd.factorize(writers_df["writer"])
writers_df["writer"] = array
wirters_list = writers_df.groupby("item")["writer"].apply(list)

In [38]:
wirters_list

item
1         [296, 320, 748, 891, 2090, 2351, 2623]
2                             [1304, 2445, 2521]
3                                         [1435]
4                                          [466]
5                       [1165, 1226, 1812, 2313]
                           ...                  
118696              [183, 579, 2484, 2491, 2570]
118900                                    [1353]
118997                                    [1576]
119141                              [2162, 2895]
119145                  [2528, 2727, 2903, 2932]
Name: writer, Length: 5648, dtype: object

In [ ]:
a = torch